# Collaboration and Competition Project
This is the report on my solution of the Collaboration and Competition Project from the [Udacity deep reinforcement learning course](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893). This is the third project from that course. It is based on the [Unity ML Tennis](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis) environment.

## Implementation notes
I used the [Facade design pattern](https://en.wikipedia.org/wiki/Facade_pattern) to create a facade between Unity ML and [OpenAI gym](https://gym.openai.com/). I created the [UnityMlFacade](unity_ml_facade.py) class. In doing this I opened up the possibilities of leveraging a number of frameworks and options to solve this problem.

I used the **MADDPG** algorithm as specified in the paper [Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments](https://arxiv.org/abs/1706.02275) by R. Lowe et al., 2017.

This in turn uses the **DDPG** algorithm as explained in the paper [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971)

### DDPG algorithm
![DDPG_algorithm.png](DDPG_algorithm.png)

### The noise process
To motivate exploration noise is added. The noise that is added is based on the **Ornstein-Uhlenbeck Process** (OU noise). Please see the reference at the end of this notebook.

`OUnoise(t) = OUnoise(t-1) + ou_theta*(asymptotic_mean - OUnoise(t-1)) + ou_sigma*Gaussian_diffusion`

This is implemented in the python script [ou_noise.py](ou_noise.py). To ensure that the amount of noise added gradually decreases over time a cumulative scaling factor of 0.995 is applied each episode. The book [Grokking Deep Reinforcement Learning](https://www.manning.com/books/grokking-deep-reinforcement-learning) has some great explorations and implementations of this noise factor.

## Experiment architecture
I have implemented an experiment based framework that allows for exploration of different hyperparameters when training a model. The parameters that you can specify are these:

- **actor-learning-rate** the learning rate to use for training the actor part of the model. Default is 0.0001.
- **critic-learning-rate** the learning rate to use for training the critic part of the model. Default is 0.001.
- **batch-size** the size of batches that are sampled and used to train the model. Default is 128.
- **buffer-size** the size of the replay buffer. Default is 100,000.
- **total-timesteps** the total number of timesteps to train for. Default is 100,000.
- **seed** random seed to use. Default is -1 which means generate a random value.
- **environment-port** this is the port number used for communication with the Unity environment. If you want to have more than one agent running at the same time you would specify a different port for each of them. Default is 5005.
- **hidden-layers** the hidden layers to use in the neural networks. Specified as a comma separated list. Default is "400,300".
- **executable-path** the path to the executable to run. Please see the notes about [setup project](Setup.md) to help with this.
- **experiments-root** the root folder that experiment output will be written to.
- **experiment-name** the name of the experiment. A subfolder will be created to the 'experiments-root' folder with this name and all output from this experiment will be written to it.
- **reward-threshold** the reward value that the solution must attain over 100 consecutive episodes. Default is 0.5.
- **gamma** the gamma value used for greedy strategy. Default is 0.99.

Please note that the default values chosen for these parameters correspond to the values used in the original [DDPG paper](https://arxiv.org/pdf/1509.02971.pdf) (see the **Experiment details** section of the paper).

## Results
I was able to use the **MADDPG** algorithm to solve this problem using the code provided in this repository.

### Solution Command Line
To reproduce the results run the [train_agent.py](train_agent.py) script as follows:

```python train_agent.py --experiments-root "D:\Data\Udacity\experiments\Tennis" --experiment-name maddpg-lr_0_0001-400_300-ts_250K --actor-learning-rate 0.0001 --critic-learning-rate 0.001 --total-timesteps 250000 --environment-port 5005 --policy-layers "400,300"```

### Solution Parameters
I used the following parameters in my solution:

- **actor-learning-rate** 0.0001
- **critic-learning-rate** 0.001
- **batch-size** 128
- **buffer-size** 100,000
- **seed** 614847 (this was a randomly generated value)
- **hidden-layers** "400,300"

As you can see from the graph below I achieved the target of 100 consecutive episodes with an average reward >= 0.5 after 2,145 episodes:

![MADDPG Results](maddpg_summary_scores.png)

### Solution Model Weights
I have included the solution model weights in this repository. They are in these files: 

- [final_actor_opponent.pth](final_actor_opponent.pth)
- [final_actor_player.pth](final_actor_player.pth)
- [final_critic_opponent.pth](final_critic_opponent.pth)
- [final_critic_player.pth](final_critic_player.pth)

## Ideas for future work

1. At the moment the player and opponent are both trained at the same time. An improvement would be to train the player first and then copy trained models to the opponent. In this way the player would be collaborating with improving versions of the opponent. This would improve training times and create better models.


## Acknowledgements

* I consulted the book [Grokking Deep Reinforcement Learning](https://www.manning.com/books/grokking-deep-reinforcement-learning) for algorithms, maths and explanations.
* The **DDPG** algorithm is explained in the paper [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971) by T. P. Lillicrap et al., 2016.
* A description of the Ornstein-Uhlenbeck Process is detailed in the textbook [Stochastic Methods, a handbook for the natural and social sciences](https://www.springer.com/gp/book/9783540707127) by Crispin Gardiner.
* I used the DDPG implementation from Udacity's GitHub repository [ddpg-pendulum](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum).
* The **MADDPG** algorithm is explained in the paper [Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments](https://arxiv.org/abs/1706.02275) by R. Lowe et al., 2017.
* I also used the MADDPG explanation/code from the drlnd course.